## Load and Preprocess

In [1]:
from sklearn import datasets
import numpy

x_sparse, y = datasets.load_svmlight_file('diabetes')
x = x_sparse.todense()

print('Shape of x: ' + str(x.shape))
print('Shape of y: ' + str(y.shape))

Shape of x: (768, 8)
Shape of y: (768,)


In [2]:
# partition the data to training and test sets
n = x.shape[0]
n_train = 640
n_test = n - n_train

rand_indices = numpy.random.permutation(n)
train_indices = rand_indices[0:n_train]
test_indices = rand_indices[n_train:n]

x_train = x[train_indices, :]
x_test = x[test_indices, :]
y_train = y[train_indices].reshape(n_train, 1)
y_test = y[test_indices].reshape(n_test, 1)

print('Shape of x_train: ' + str(x_train.shape))
print('Shape of x_test: ' + str(x_test.shape))
print('Shape of y_train: ' + str(y_train.shape))
print('Shape of y_test: ' + str(y_test.shape))

Shape of x_train: (640, 8)
Shape of x_test: (128, 8)
Shape of y_train: (640, 1)
Shape of y_test: (128, 1)


In [3]:
# Standardization
import numpy

# calculate mu and sig using the training set
d = x_train.shape[1]
mu = numpy.mean(x_train, axis=0).reshape(1, d)
sig = numpy.std(x_train, axis=0).reshape(1, d)

# transform the training features
x_train = (x_train - mu) / (sig + 1E-6)

# transform the test features
x_test = (x_test - mu) / (sig + 1E-6)

print('test mean = ')
print(numpy.mean(x_test, axis=0))

print('test std = ')
print(numpy.std(x_test, axis=0))

test mean = 
[[-0.17272232 -0.06402743 -0.13467415 -0.01027226  0.01422262  0.06318235
  -0.02004334  0.00417023]]
test std = 
[[0.97501882 1.12457975 1.06447313 0.93029679 1.0017629  0.9233102
  1.06444755 1.06905814]]


In [4]:
n_train, d = x_train.shape
x_train = numpy.concatenate((x_train, numpy.ones((n_train, 1))), axis=1)

n_test, d = x_test.shape
x_test = numpy.concatenate((x_test, numpy.ones((n_test, 1))), axis=1)

print('Shape of x_train: ' + str(x_train.shape))
print('Shape of x_test: ' + str(x_test.shape))

Shape of x_train: (640, 9)
Shape of x_test: (128, 9)


## Stochastic GD

In [5]:
def stochastic_objective_gradient(w, xi, yi, lam):
    d = xi.shape[0]
    yx = yi * xi # 1-by-d matrix
    yxw = float(numpy.dot(yx, w)) # scalar
    
    # calculate objective function Q_i
    loss = numpy.log(1 + numpy.exp(-yxw)) # scalar
    reg = lam / 2 * numpy.sum(w * w) # scalar
    obj = loss + reg
    
    # calculate stochastic gradient
    g_loss = -yx.T / (1 + numpy.exp(yxw)) # d-by-1 matrix
    g = g_loss + lam * w # d-by-1 matrix
    
    return obj, g

In [6]:
n,d = x_train.shape
lam = 1E-6
xi = x_train[0, :] # 1-by-d matrix
yi = float(y_train[0, :]) # scalar
w = numpy.zeros((d, 1))
xi, yi, w

(matrix([[ 2.09165346, -1.15889146,  0.23369685, -1.27540799, -0.69073359,
          -0.22677074, -0.52816416,  0.15214607,  1.        ]]),
 1.0,
 array([[0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.]]))

In [7]:
stochastic_objective_gradient(w, xi, yi, lam)

(0.6931471805599453,
 matrix([[-1.04582673],
         [ 0.57944573],
         [-0.11684842],
         [ 0.63770399],
         [ 0.3453668 ],
         [ 0.11338537],
         [ 0.26408208],
         [-0.07607304],
         [-0.5       ]]))

In [8]:
yx = yi * xi # 1-by-d matrix
yxw = float(numpy.dot(yx, w)) # scalar
yx, yxw

(matrix([[ 2.09165346, -1.15889146,  0.23369685, -1.27540799, -0.69073359,
          -0.22677074, -0.52816416,  0.15214607,  1.        ]]),
 0.0)

In [9]:
loss = numpy.log(1 + numpy.exp(-yxw)) # scalar
reg = lam / 2 * numpy.sum(w * w) # scalar
obj = loss + reg
loss, reg, obj

(0.6931471805599453, 0.0, 0.6931471805599453)

In [10]:
z = 1 / (1 + numpy.exp(yxw))
z

0.5

In [11]:
g_loss = -yx.T / (1 + numpy.exp(yxw)) # d-by-1 matrix
g_loss_2 = z * -yx.T
g = g_loss + lam * w # d-by-1 matrix
print(numpy.array_equal(g_loss, g_loss_2))
g_loss, g

True


(matrix([[-1.04582673],
         [ 0.57944573],
         [-0.11684842],
         [ 0.63770399],
         [ 0.3453668 ],
         [ 0.11338537],
         [ 0.26408208],
         [-0.07607304],
         [-0.5       ]]),
 matrix([[-1.04582673],
         [ 0.57944573],
         [-0.11684842],
         [ 0.63770399],
         [ 0.3453668 ],
         [ 0.11338537],
         [ 0.26408208],
         [-0.07607304],
         [-0.5       ]]))

In [58]:
objvals = numpy.zeros(8)
mat = numpy.zeros((8,9))
for i in range(8):
    n,d = x_train.shape
    lam = 1E-6
    xi = x_train[i, :] # 1-by-d matrix
    yi = float(y_train[i, :]) # scalar
    w = numpy.zeros((d, 1))
    obj, g = stochastic_objective_gradient(w, xi, yi, lam)
    objvals[i] = obj
    mat[i, :] = g.T
objavg = numpy.average(objvals)
w_star = numpy.average(mat, axis=0).reshape((9,1))
objavg, w_star

(0.6931471805599453,
 array([[-0.02494749],
        [ 0.43858289],
        [ 0.03665312],
        [ 0.18755359],
        [ 0.22536851],
        [ 0.1963183 ],
        [ 0.20493193],
        [ 0.22275409],
        [-0.375     ]]))

## Mini-batch SGD

In [47]:
def mb_stochastic_objective_gradient(w, xi, yi, lam, b):
    # Fill the function
    # Follow the implementation of stochastic_objective_gradient
    # Use matrix-vector multiplication; do not use FOR LOOP of vector-vector multiplications
    n,d = xi.shape
    yx = numpy.multiply(yi, xi)  # b by d matrix
    yxw = numpy.dot(yx, w)  # b x 1 vector

    # Solve objective function Q_i
    loss = numpy.log(1 + numpy.exp(-yxw)) # b x 1 vector
    reg = (lam / 2) * numpy.sum(w * w)  # scalar
    obj = numpy.average(loss + reg)  # scalar

    # Calculate gradient
    z = (1 / (1 + numpy.exp(yxw)))
    g_loss = numpy.multiply(z, -yx)
    reg_loss = g_loss.T + (lam * w) # d x n matrix
    g = numpy.average(reg_loss, axis=1) # d x 1 vector
    return obj, g

In [60]:
lam = 1E-6
b = 8
xi = x_train[0:b, :]
yi = y_train[0:b].reshape((b, 1))
w = numpy.zeros((d,1))
obj, g = mb_stochastic_objective_gradient(w, xi, yi, lam, b)
obj == objavg, numpy.array_equal(g, w_star)

(True, True)

In [57]:
obj, g = mb_stochastic_objective_gradient(w, xi, yi, lam, b)
g, numpy.array_equal(g, avg)

(matrix([[-0.02494749],
         [ 0.43858289],
         [ 0.03665312],
         [ 0.18755359],
         [ 0.22536851],
         [ 0.1963183 ],
         [ 0.20493193],
         [ 0.22275409],
         [-0.375     ]]),
 True)

In [34]:
yx = numpy.multiply(yi, xi)
yxw = numpy.dot(yx, w)
yx, yxw

(matrix([[ 2.09165346, -1.15889146,  0.23369685, -1.27540799, -0.69073359,
          -0.22677074, -0.52816416,  0.15214607,  1.        ],
         [-0.57558673,  0.63247704, -0.60362146,  0.83331307,  0.42101948,
          -0.81608078,  0.69018075, -0.79489933,  1.        ],
         [ 0.31349334,  0.0246913 ,  0.86168558, -1.27540799, -0.69073359,
           0.35000092, -0.55869912, -0.020044  ,  1.        ],
         [-0.57558673, -0.9349704 , -0.91761582, -1.27540799, -0.69073359,
          -0.22677074, -1.01367003, -0.9670894 ,  1.        ],
         [ 0.01713331, -0.35917338,  0.33836164, -0.03498384,  0.17782349,
          -0.43992543, -1.08390044, -0.53661422,  1.        ],
         [-0.01713331, -2.00799213, -0.44302642, -1.14341911, -1.71516953,
          -0.63838656,  0.63809002,  0.19223407, -1.        ],
         [-0.57558673, -1.70269976,  0.02436727,  0.70927065, -0.11748592,
          -0.86623482, -0.87320921, -0.70880429,  1.        ],
         [-0.27922671, -1.51076742

In [35]:
loss = numpy.log(1 + numpy.exp(-yxw))
reg = lam / 2 * numpy.sum(w * w)
obj = numpy.average(loss + reg)  # scalar
loss, reg, obj

(matrix([[0.69314718],
         [0.69314718],
         [0.69314718],
         [0.69314718],
         [0.69314718],
         [0.69314718],
         [0.69314718],
         [0.69314718]]),
 0.0,
 0.6931471805599453)

In [36]:
z = 1 / (1 + numpy.exp(yxw))
z

matrix([[0.5],
        [0.5],
        [0.5],
        [0.5],
        [0.5],
        [0.5],
        [0.5],
        [0.5]])

In [37]:
g_loss = numpy.multiply(z, -yx)
g_loss

matrix([[-1.04582673,  0.57944573, -0.11684842,  0.63770399,  0.3453668 ,
          0.11338537,  0.26408208, -0.07607304, -0.5       ],
        [ 0.28779336, -0.31623852,  0.30181073, -0.41665653, -0.21050974,
          0.40804039, -0.34509038,  0.39744967, -0.5       ],
        [-0.15674667, -0.01234565, -0.43084279,  0.63770399,  0.3453668 ,
         -0.17500046,  0.27934956,  0.010022  , -0.5       ],
        [ 0.28779336,  0.4674852 ,  0.45880791,  0.63770399,  0.3453668 ,
          0.11338537,  0.50683501,  0.4835447 , -0.5       ],
        [-0.00856666,  0.17958669, -0.16918082,  0.01749192, -0.08891175,
          0.21996272,  0.54195022,  0.26830711, -0.5       ],
        [ 0.00856666,  1.00399607,  0.22151321,  0.57170955,  0.85758477,
          0.31919328, -0.31904501, -0.09611704,  0.5       ],
        [ 0.28779336,  0.85134988, -0.01218364, -0.35463533,  0.05874296,
          0.43311741,  0.43660461,  0.35440215, -0.5       ],
        [ 0.13961335,  0.75538371,  0.04014876, 

In [38]:
g_loss = g_loss.T + lam * w
numpy.array_equal(g_loss.T, mat)

True

In [20]:
g = numpy.average(g_loss, axis=1)
g1 = (-0.42018124,  0.02528976,  0.17378009,  0.27169091, -0.12320057, -0.02528976,  0.47076076, -0.17378009)
g1 = sum(g1) / 8
g, g1

(matrix([[-0.02494749],
         [ 0.43858289],
         [ 0.03665312],
         [ 0.18755359],
         [ 0.22536851],
         [ 0.1963183 ],
         [ 0.20493193],
         [ 0.22275409],
         [-0.375     ]]),
 0.0248837325)

In [21]:
iters = int(n / b)
start = 0
for i in range(iters):
    end = start + b
    print(start, end)
    xi = x[rand_indices[0:b], :]
    print(len(xi))
    yi = y[rand_indices[0:b]].reshape((b, 1))
    start = end

0 8
8
8 16
8
16 24
8
24 32
8
32 40
8
40 48
8
48 56
8
56 64
8
64 72
8
72 80
8
80 88
8
88 96
8
96 104
8
104 112
8
112 120
8
120 128
8
128 136
8
136 144
8
144 152
8
152 160
8
160 168
8
168 176
8
176 184
8
184 192
8
192 200
8
200 208
8
208 216
8
216 224
8
224 232
8
232 240
8
240 248
8
248 256
8
256 264
8
264 272
8
272 280
8
280 288
8
288 296
8
296 304
8
304 312
8
312 320
8
320 328
8
328 336
8
336 344
8
344 352
8
352 360
8
360 368
8
368 376
8
376 384
8
384 392
8
392 400
8
400 408
8
408 416
8
416 424
8
424 432
8
432 440
8
440 448
8
448 456
8
456 464
8
464 472
8
472 480
8
480 488
8
488 496
8
496 504
8
504 512
8
512 520
8
520 528
8
528 536
8
536 544
8
544 552
8
552 560
8
560 568
8
568 576
8
576 584
8
584 592
8
592 600
8
600 608
8
608 616
8
616 624
8
624 632
8
632 640
8
